# MS MARCO

In [1]:
# Commom imports
from tqdm import tqdm

## Dataset

In [2]:
# load data, tokenize and split
from src.datasets import MSMarcoDataset

dataset = MSMarcoDataset('data/subset_msmarco_train_0')
dataset.get_data('subset_msmarco_train_0.01_99.pkl')

Loading data from subset_msmarco_train_0.01_99.pkl...


Loading qrels: 100%|██████████| 2845/2845 [00:00<00:00, 947347.96it/s]
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\esdra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esdra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing data...


Tokenizing documents: 100%|██████████| 277168/277168 [00:58<00:00, 4778.49it/s]

Split data into train and test sets...
Data loading and processing complete.


## Algorithms

### Retrivers

#### BM25

In [3]:
from src.retrivers.bm25 import BM25

bm25 = BM25(dataset.documents)

docs = bm25.get_top_n(dataset.queries['135841'].tokenized_text, 100)

print('True relevant documents for query 135841:')
for doc_id in dataset.qrels['135841'].docs_ids:
    for i, (doc, score) in enumerate(docs):
        if doc_id == doc:
            print(f'{i+1}º: {doc}')
            break
docs[:10]

True relevant documents for query 135841:
29º: msmarco_passage_16_93510573


[('msmarco_passage_02_20739671', 27.412804036648346),
 ('msmarco_passage_02_19009299', 24.213466338904247),
 ('msmarco_passage_02_18753979', 22.453242048560078),
 ('msmarco_passage_00_53431480', 21.618148811078974),
 ('msmarco_passage_03_547575435', 21.50421455547894),
 ('msmarco_passage_02_19817427', 20.711329688508723),
 ('msmarco_passage_02_21092853', 20.654825487373298),
 ('msmarco_passage_02_26827419', 20.654825487373298),
 ('msmarco_passage_03_18532150', 20.415866982320434),
 ('msmarco_passage_04_550536616', 20.20047378077205)]

##### Evaluation

In [4]:
# Evaluating BM25 with MRR and NDCG
from src.metrics import mmr_score

sorted_docs = {}
for query_id in tqdm(dataset.test_query_ids, desc="Evaluating BM25"):
    # Get the top 10 documents for the query
    docs = bm25.get_top_n(dataset.queries[query_id].tokenized_text, 100)
    sorted_docs[query_id] = docs

mrr = mmr_score(sorted_docs, dataset.qrels)
print(f"MRR: {mrr:.4f}")

Evaluating BM25: 100%|██████████| 555/555 [08:32<00:00,  1.08it/s]

MRR: 0.4723


### Rerankers

#### MonoBERT

In [5]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Usando o modelo pré-treinado "bert-base-uncased" como exemplo.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
monobert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

print("Modelo monoBERT (bert-base-uncased) e tokenizer carregados com sucesso!")

c:\Users\esdra\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (c:\Users\esdra\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\framework\dtypes.py)

In [ ]:
def score_with_monobert(query, doc_texts, max_length=512):
    """
    Para cada par (query, documento), gera uma pontuação de relevância usando o monoBERT.
    
    Args:
        query (str): Texto da consulta.
        doc_texts (list of str): Lista contendo os textos dos documentos.
        max_length (int): Comprimento máximo dos tokens (default=512).
    
    Returns:
        scores (list of float): Lista de pontuações para cada documento.
    """
    scores = []
    for doc in doc_texts:
        # Prepara a entrada concatenando a query e o documento.
        inputs = tokenizer.encode_plus(
            query,
            doc,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_tensors="tf"
        )
        # Executa o modelo em modo inference
        outputs = monobert_model(inputs)
        logits = outputs.logits  # formato: [batch_size, num_labels]
        # Supondo que a classe 1 representa relevância
        relevance_score = logits[0][1].numpy()  
        scores.append(relevance_score)
    return scores


In [ ]:
# Defina um query_id para exemplificar (ajuste conforme seu dataset)
query_id = '135841'

# Recupera os candidatos usando BM25 (por exemplo, os top 100)
# bm25_results agora é uma lista de tuplas: (doc_id, BM25_score)
bm25_results = bm25.get_top_n(dataset.queries[query_id].tokenized_text, 100)

# Obtém os textos dos documentos correspondentes extraindo o doc_id de cada tupla
bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, score in bm25_results]

# Recupera o texto completo da query
query_text = dataset.queries[query_id].text

# Calcula as pontuações de relevância usando monoBERT
monobert_scores = score_with_monobert(query_text, bm25_doc_texts)

# Cria uma lista apenas com os doc_ids (primeiro elemento de cada tupla)
doc_ids = [doc_id for doc_id, score in bm25_results]

# Combina os IDs dos documentos com as pontuações do monoBERT
doc_score_pairs = list(zip(doc_ids, monobert_scores))

# Reordena os documentos de acordo com a pontuação (maior indica maior relevância)
doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

# Seleciona os top 10 documentos após reordenação
top10_monobert = doc_score_pairs_sorted[:10]

print("Top 10 documentos reordenados (BM25 + monoBERT):")
for rank, (doc_id, score) in enumerate(top10_monobert, start=1):
    print(f"Rank {rank}: Documento ID {doc_id} | Pontuação: {score:.4f}")


In [ ]:
# Célula de Visualização: Exibe a query e os top 10 documentos com seus textos e pontuações

# Usando o mesmo query_id de exemplo
query_id = '135841'
query_text = dataset.queries[query_id].text

# Recupera os candidatos via BM25 (top 100) e seus textos, calculando as pontuações do monoBERT
bm25_results = bm25.get_top_n(dataset.queries[query_id].tokenized_text, 100)
bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, _ in bm25_results]
monobert_scores = score_with_monobert(query_text, bm25_doc_texts)
doc_ids = [doc_id for doc_id, _ in bm25_results]
doc_score_pairs = list(zip(doc_ids, monobert_scores))
doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
top10_monobert = doc_score_pairs_sorted[:10]

# Exibe a query e os top 10 documentos com um trecho do texto
print("Query:", query_text)
print("\nTop 10 documentos reordenados (BM25 + monoBERT):")
for rank, (doc_id, score) in enumerate(top10_monobert, start=1):
    # Obtem o texto completo do documento e limita a um snippet (ex: 300 caracteres)
    doc_text = dataset.documents[doc_id].text
    snippet = doc_text[:300] + "..." if len(doc_text) > 300 else doc_text
    print(f"Rank {rank}: Documento ID {doc_id} | Pontuação: {score:.4f}")
    print("Trecho do documento:", snippet)
    print("-" * 80)


In [ ]:
# Célula 7: Avaliação com Métricas (MRR e NDCG) usando a arquitetura Multirank
sorted_docs = {}

# Percorre cada consulta do conjunto de teste (isso pode demorar, pois invoca o modelo BERT para cada consulta)
for q_id in tqdm(dataset.test_query_ids, desc="Reordenando queries com monoBERT"):
    # Recupera os candidatos via BM25 (por exemplo, os top 100)
    bm25_results = bm25.get_top_n(dataset.queries[q_id].tokenized_text, 100)
    # bm25_results é uma lista de tuplas (doc_id, BM25_score)
    # Extraímos os textos dos documentos
    bm25_doc_texts = [dataset.documents[doc_id].text for doc_id, score in bm25_results]
    # Recupera o texto da consulta
    query_txt = dataset.queries[q_id].text
    # Calcula as pontuações de relevância com monoBERT para cada candidato
    monobert_scores = score_with_monobert(query_txt, bm25_doc_texts)
    # Extrai apenas os doc_ids
    doc_ids = [doc_id for doc_id, score in bm25_results]
    # Associa cada doc_id à pontuação do monoBERT
    doc_score_pairs = list(zip(doc_ids, monobert_scores))
    # Ordena os documentos em ordem decrescente de pontuação
    doc_score_pairs_sorted = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    # Seleciona os top 10 documentos para a consulta
    sorted_docs[q_id] = [doc_id for doc_id, score in doc_score_pairs_sorted[:10]]

# Calcula as métricas usando as funções importadas
mrr = mmr_score(sorted_docs, dataset.qrels)
ndcg = ndcg_score(sorted_docs, dataset.qrels)
print(f"MRR: {mrr:.4f}")
print(f"NDCG: {ndcg:.4f}")